In [147]:
import pandas as pd
import numpy as np
import mysql.connector
import matplotlib.pyplot as plt
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import en_core_web_sm
import joblib

In [3]:
# Buka Koneksi ke MySQL

connection = mysql.connector.connect(host = 'localhost', port = 3306, user = 'Luthfiaed', passwd = '*', database = 'final_project')
database = connection.cursor(dictionary=True)

In [ ]:
# Ambil data dari MySQL ke Pandas DataFrame
query = 'SELECT * FROM imdbMovies;'
database.execute(query)
data = database.fetchall()

dfImdb = pd.DataFrame(data)

In [135]:
# Bersihkan data
dfImdbEng = dfImdb[dfImdb.language.astype('str').str.contains("English")] 
dfImdb1990 = dfImdbEng[dfImdb.year >= 1990]
dfImdb_clean = dfImdb1990[dfImdb.description.notnull()]
dfImdb_clean.title = dfImdb_clean.title.apply(lambda x: x.lower())

dfImdb_clean.shape

<ipython-input-135-717e82c29d04>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfImdb1990 = dfImdbEng[dfImdb.year >= 1990]
<ipython-input-135-717e82c29d04>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfImdb_clean = dfImdb1990[dfImdb.description.notnull()]
c:\users\luthf\appdata\local\programs\python\python38\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


(28257, 22)

In [136]:
# Tokenization & Lemmatization deskripsi film
nlp = en_core_web_sm.load()
stopwords = spacy.lang.en.stop_words.STOP_WORDS

listOfLemmas = []
for desc in dfImdb_clean.description :
    tokens = nlp(desc)
    lemmas = [token.lemma_ for token in tokens]
    lemmas_clean = [lemma for lemma in lemmas if lemma.isalpha() and lemma not in stopwords]
    lemmas_string = " ".join(lemmas_clean)
    listOfLemmas.append(lemmas_string)

In [137]:
# Buat model TfIdf

tfidf_vectorizer = TfidfVectorizer(
#                                    max_df=0.8, 
#                                    max_features=200000, 
#                                    min_df=0.2, 
#                                    stop_words='english', 
#                                    use_idf=True, 
#                                    tokenizer=tokenize_and_stem, 
#                                    ngram_range=(1,3)
                                  )

tfIdfMatrix = tfidf_vectorizer.fit_transform([doc for doc in listOfLemmas])
tfIdfMatrix = tfIdfMatrix.astype(np.float32)

In [138]:
# Hitung skor kemiripan film

cos_similarity = cosine_similarity(tfIdfMatrix)
similarity_distance = 1 - cos_similarity

In [139]:
# Fungsi rekomendasi

def recommend(film) :                     
    filmIndex = dfImdb_clean.index.get_loc(dfImdb_clean[dfImdb_clean.title == film].index[0])
    similarMovies = list(enumerate(similarity_distance[filmIndex]))
    similarMovies = [i for i in similarMovies if i[0] != filmIndex]
    similarMovies = sorted(similarMovies, key=lambda x: x[1], reverse=False)

    similarMoviesList = []
    
    for i in similarMovies[:5]:
        similarMoviesList.append(dfImdb_clean.iloc[i[0]])
    
    dfSimilarMovies = pd.DataFrame(similarMoviesList)
    
    return [i for i in dfSimilarMovies.index]

In [144]:
joblib.dump(similarity_distance, 'simDistance')

['simDistance']

In [146]:
joblib.dump(dfImdb_clean, 'dfImdb_clean')

['dfImdb_clean']